In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torchvision.models as models
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import torch.nn.functional as F
import torch.multiprocessing as mp
import json
import ray

In [2]:
transform_all = T.Compose([
    T.Resize((224, 224)),   # paksa seragam
    T.ToTensor(),
])

In [3]:
data_root = "data"   # Ubah sesuai directory Anda
full_dataset = ImageFolder(data_root, transform=transform_all)
num_classes = len(full_dataset.classes)

# split dataset otomatis
train_size = int(0.7 * len(full_dataset))
val_size   = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [4]:
class ResNetBottleneck(nn.Module):
    def __init__(self, in_channels, filters):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, filters, 1)
        self.bn1 = nn.BatchNorm2d(filters)

        self.conv2 = nn.Conv2d(filters, filters, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(filters)

        self.conv3 = nn.Conv2d(filters, filters * 4, 1)
        self.bn3 = nn.BatchNorm2d(filters * 4)

        self.shortcut_needed = (in_channels != filters * 4)
        if self.shortcut_needed:
            self.shortcut_conv = nn.Conv2d(in_channels, filters * 4, 1)

    def forward(self, x):
        shortcut = x
        if self.shortcut_needed:
            shortcut = self.shortcut_conv(x)

        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))

        return torch.relu(x + shortcut)


class HybridEfficientNet(nn.Module):
    def __init__(self, num_classes=4, dropoout=0.3):
        super().__init__()

        # Load pretrained EfficientNet-B0
        self.base = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

        # Freeze feature extractor
        self.base.features.requires_grad_(False)

        #in_channels = 1280  # output EfficientNetB0 channels
        in_channels = self.base.features[-1].out_channels

        self.b1 = ResNetBottleneck(in_channels, 64)
        self.b2 = ResNetBottleneck(64 * 4, 64)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropoout)
        self.fc = nn.Linear(64 * 4, num_classes)

    def forward(self, x):
        x = self.base.features(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

class HybridMobileNet(nn.Module):
    def __init__(self, num_classes=4, dropout=0.3):
        super().__init__()

        self.base = models.mobilenet_v3_small(
            weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
        )

        self.base.features.requires_grad_(False)

        # ambil channel output terakhir
        in_channels = self.base.features[-1].out_channels  # = 576

        self.b1 = ResNetBottleneck(in_channels, 64)   # 576 → 256
        self.b2 = ResNetBottleneck(64 * 4, 64)        # 256 → 256

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(64 * 4, num_classes)      # 256 → num_classes

    def forward(self, x):
        x = self.base.features(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

In [5]:
def train_model(device, model, optimizer, name="model", scheduler=False, epochs=10):
    criterion = nn.CrossEntropyLoss()
    
    scheduler_optimizer = None
    if scheduler:
        scheduler_optimizer = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=epochs
        )
    
    start_time = time.perf_counter()

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        train_accs.append(correct / total)

        model.eval()
        val_running_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)

                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_accs.append(correct / total)

        if scheduler_optimizer is not None:
            scheduler_optimizer.step() 

        print(f"[{name}] Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"  Train Acc : {train_accs[-1]:.4f} | Val Acc : {val_accs[-1]:.4f}")

        clear_output(wait=True)

        fig, ax = plt.subplots(1, 2, figsize=(12, 5))

        # Loss panel
        ax[0].plot(train_losses, label="Train Loss")
        ax[0].plot(val_losses, label="Val Loss")
        ax[0].set_title("Training & Validation Loss")
        ax[0].legend()

        # Acc panel
        ax[1].plot(train_accs, label="Train Acc")
        ax[1].plot(val_accs, label="Val Acc")
        ax[1].set_title("Training & Validation Accuracy")
        ax[1].legend()

        plt.tight_layout()
        display(fig)
        plt.close(fig)

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    return train_losses, val_losses, train_accs, val_accs, elapsed_time

def Summary(name, epoch, train_losses, val_losses, train_accs, val_accs, elapsed_time):
    print()
    print(f"[{name}]")
    print(f"  Total Epochs: {epoch}")
    print(f"  Train Loss: {np.average(train_losses):.4f} | Val Loss: {np.average(val_losses):.4f}")
    print(f"  Train Acc : {train_accs[-1]:.4f} | Val Acc : {val_accs[-1]:.4f}")
    print(f"  Time : {elapsed_time:.4f}s ~ {(elapsed_time / 60):.4f}m")

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import torch

import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, class_names, figsize=(8,6)):
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion Matrix")
    plt.colorbar()

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Label angka di kotak
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="black" if cm[i, j] > thresh else "white")

    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.tight_layout()
    plt.show()

def classification_report_df(y_true, y_pred, class_names):
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    df = pd.DataFrame(report).transpose()
    return df

def evaluate_model(model, dataloader, device, class_names):
    start_time = time.perf_counter()
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    cm = confusion_matrix(all_labels, all_preds)

    return {
        "confusion_matrix": cm,
        "preds": all_preds,
        "labels": all_labels,
        "time": elapsed_time
    }

def clean_gpu():
    for var in list(globals().keys()):
        if isinstance(globals()[var], torch.Tensor):
            del globals()[var]

    import gc
    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

In [6]:
# Hyperparameter bounds
LR_MIN, LR_MAX = 0.003134621711988863, 0.004174003510706967
WD_MIN, WD_MAX = 0.0002606683474969785, 0.0009743301033360753
DP_MIN, DP_MAX = 0.21222920791578329, 0.4609636959573514

#global name, model
name = "mobresnet"
#model = HybridMobileNet(num_classes=4)

In [7]:
ray.init(ignore_reinit_error=True)
from IPython.display import display

@ray.remote(num_gpus=1)
def train_remote(params):
    #def display(*args, **kwargs):
    #    pass
        
    lr, wd, dp = params

    # Inisialisasi model di worker
    model = HybridMobileNet(num_classes=4)
    device = torch.device("cuda")

    model.dropout.p = dp
    model = model.to(device)

    lr = abs(lr)
    wd = abs(wd)
    dp = abs(dp)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

    train_losses, val_losses, train_accs, val_accs, _ = train_model(
        device, model, optimizer, name=name, scheduler=True, epochs=10
    )

    fitness = val_accs[-1]

    torch.save(model.state_dict(), "./ga/" + name + "-" + str(fitness) + ".pth")

    del model
    del optimizer
    clean_gpu()

    return fitness

2025-12-10 17:52:12,600	INFO worker.py:2023 -- Started a local Ray instance.
C:\Users\Gia\anaconda3\envs\torch-gpu\lib\site-packages\ray\_private\worker.py:2062: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
(pid=gcs_server) [2025-12-10 17:52:42,292 E 10956 1644] (gcs_server.exe) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-12-10 17:52:44,675 E 25940 22368] (raylet.exe) main.cc:979: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 1

(train_remote pid=10824) [mobresnet] Epoch 1/10
(train_remote pid=10824)   Train Loss: 0.3127 | Val Loss: 0.3751
(train_remote pid=10824)   Train Acc : 0.8825 | Val Acc : 0.8550
m(train_remote pid=10824) 
m(train_remote pid=8360) 
(train_remote pid=10824) Figure(1200x500)
(train_remote pid=8360) Figure(1200x500)
m(train_remote pid=8360) 
(train_remote pid=8360) [mobresnet] Epoch 2/10 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_remote pid=8360)   Train Loss: 0.1545 | Val Loss: 0.6195 [repeated 2x across cluster]
(train_remote pid=8360)   Train Acc : 0.9450 | Val Acc : 0.7940 [repeated 2x across cluster]
(train_remote pid=8360) Figure(1200x500)
m(train_remote pid=10824) 
(train_remote pid=10824) Figure(1200x500)
m(train_remote pid=8360) 
(train_remote pid=8360) [mobresnet] Epoch 3/10 [r

In [8]:
def save_row(result, filename):
    # Jadikan DataFrame 1 baris
    df = pd.DataFrame([result])

    # Jika file tidak ada → buat baru dengan header
    if not os.path.exists(filename):
        df.to_csv(filename, index=False)
    else:
        # Jika ada → append tanpa header
        df.to_csv(filename, mode="a", header=False, index=False)

    print(f"Row saved to {filename}")

In [9]:
from deap import base, creator, tools
import random
import multiprocessing
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("lr", random.uniform, LR_MIN, LR_MAX)
toolbox.register("wd", random.uniform, WD_MIN, WD_MAX)
toolbox.register("dp", random.uniform, DP_MIN, DP_MAX)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.lr, toolbox.wd, toolbox.dp), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.05, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

def evaluate_population(pop, generation):
    futures = []

    for i, ind in enumerate(pop):
        name = f"ga_gen{generation}_ind{i}"
        futures.append(train_remote.remote(ind))

    results = ray.get(futures)

    for ind, fit in zip(pop, results):
        ind.fitness.values = (fit,)

        flat = {
            "generation": generation,
            "learning_rate": ind[0],
            "weight_decay": ind[1],
            "dropout": ind[2],
            "fitness": fit,
        }

        # panggil save_row()
        save_row(flat, "ga_results.csv")


toolbox.register("evaluate", evaluate_population)

def run_ga(n_gen=5, pop_size=10):
    pop = toolbox.population(pop_size)

    for gen in range(n_gen):
        print(f"\n===== GENERATION {gen+1} =====")

        # Evaluasi (parallel GPU)
        evaluate_population(pop, gen)

        # Seleksi
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Crossover
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.9:
                tools.cxBlend(c1, c2, alpha=0.4)

        # Mutasi
        for mutant in offspring:
            if random.random() < 0.2:
                tools.mutGaussian(mutant, mu=0, sigma=0.1, indpb=0.3)

        pop = offspring

        best = tools.selBest(pop, 1)[0]
        print("Best:", best, "Fitness:", best.fitness.values[0])

    result = {
        "accuracy": float(best.fitness.values[0]),
        "best_parameters": best,
    }

    # Simpan ke file JSON
    save_path = "./" + name + "-ga.json"
    with open(save_path, "w") as f:
        json.dump(result, f, indent=4)

    print(f"[GA] Hasil terbaik disimpan di {save_path}")

    return tools.selBest(pop, 1)[0]

In [10]:
start_time = time.perf_counter()

best_ga = run_ga(n_gen=10, pop_size=6)

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print("Best GA params =", best_ga)
print("Time =", elapsed_time)


===== GENERATION 1 =====
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Best: [0.04792077080109938, -0.0023629887080827204, 0.30155475178420715] Fitness: 0.9796421961752005

===== GENERATION 2 =====
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Best: [0.0031350438769153005, 0.000780812630957079, 0.3086201155309566] Fitness: 0.974090067859346

===== GENERATION 3 =====
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Row saved to ga_results.csv
Best: [0.0031350438769153005, 0.000780812630957079, 0.30930543591768817] Fitness: 0.9753238741517581

===== GENERATION 4 =====


RayTaskError(RuntimeError): [36mray::train_remote()[39m (pid=12784, ip=127.0.0.1)
  File "python\\ray\\_raylet.pyx", line 1722, in ray._raylet.execute_task
  File "C:\Users\Gia\AppData\Local\Temp\ipykernel_27764\1159917213.py", line 23, in train_remote
  File "C:\Users\Gia\AppData\Local\Temp\ipykernel_27764\3441081241.py", line 23, in train_model
  File "C:\Users\Gia\anaconda3\envs\torch-gpu\lib\site-packages\torch\utils\data\dataloader.py", line 732, in __next__
    data = self._next_data()
  File "C:\Users\Gia\anaconda3\envs\torch-gpu\lib\site-packages\torch\utils\data\dataloader.py", line 1506, in _next_data
    return self._process_data(data, worker_id)
  File "C:\Users\Gia\anaconda3\envs\torch-gpu\lib\site-packages\torch\utils\data\dataloader.py", line 1541, in _process_data
    data.reraise()
  File "C:\Users\Gia\anaconda3\envs\torch-gpu\lib\site-packages\torch\_utils.py", line 769, in reraise
    raise exception
RuntimeError: Caught RuntimeError in DataLoader worker process 0.